Your Grant Writer Reflection Agent code is ready — you’ll find it structured to:

Generate a response to a grant question

Critique it using multiple criteria

Parse the score from critique

Retry until the answer scores ≥ 8 or ends

In [ ]:
!pip install langchain
!pip install openai
!pip install langchain-community langchain

In [ ]:
!pip install langgraph
!pip install -U langchain langchain-openai

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder

In [4]:
import os

os.environ["OPENAI_API_KEY"] = 'OPENAI_API_KEY'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = 'LANGCHAIN_API_KEY'
os.environ["LANGCHAIN_PROJECT"] = "Langgraph_1"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

                            ┌────────────────────────┐
                            │   Entry Point          │
                            │   (HumanMessage: Qn)   │
                            └──────────┬─────────────┘
                                       │
                                       ▼
                           ┌───────────────────────┐
                           │       GENERATE        │
                           │  ↳ generation_chain    │
                           │  (writes the answer)   │
                           └──────────┬────────────┘
                                      │
                    ┌────────────────▼────────────────┐
                    │ Evaluate with `should_continue` │
                    │  ↳ parse_score() from response  │
                    └────────────────┬────────────────┘
                                     │
                 ┌───────────────────┴───────────────────┐
        Score ≥ 8│                                       │Score < 8
                 ▼                                       ▼
        ┌────────────────┐                    ┌────────────────────┐
        │      END       │                    │      REFLECT       │
        │ (graph stops)  │                    │ ↳ reflection_chain │
        └────────────────┘                    │ (returns critique) │
                                              └──────────┬─────────┘
                                                         │
                                                         ▼
                                                ┌────────────────┐
                                                │    GENERATE    │
                                                │   (again...)   │
                                                └────────────────┘


In [16]:
from langgraph.graph import END, MessageGraph
from langchain_core.messages import HumanMessage

In [ ]:
llm = ChatOpenAI()

In [7]:
#Define Prompts
generation_prompt = ChatPromptTemplate.from_messages([
    "system","You are a grant application writer. Write a clear, persuasive answer to a given question",
    MessagesPlaceholder(variable_name="messages")
])

reflection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a critical reviewer. Evaluate the following grant application response on:
    - Relevance to the question
    - Clarity and structure
    - Persuasiveness and tone
    Give a score out of 10 and suggestions for improvement."""),
    MessagesPlaceholder(variable_name="messages")
])

In [8]:
#Chain setup
generation_chain = generation_prompt|llm
reflection_chain = reflection_prompt|llm

In [21]:
# Score extractor
import re

def parse_score(text):
    match = re.search(r"score\s*[:\-]?\s*(\d+(?:\.\d+)?)", text, re.I)
    return float(match.group(1)) if match else 0.0

In [27]:
#Nodes

def generate_node(state):
  return generation_chain.invoke({"messages":state})

def reflect_node(messages):
  response = reflection_chain.invoke({"messages": messages})
  return [HumanMessage(content=response.content)]

def should_continue(state):
    reflections = [msg.content for msg in state if isinstance(msg, HumanMessage)]
    last = reflections[-1] if reflections else ""
    score = parse_score(last)
    if score >= 8:
        return END
    return "REFLECT"

In [28]:
# LangGraph assembly
graph = MessageGraph()
graph.add_node("GENERATE", generate_node)
graph.add_node("REFLECT", reflect_node)
graph.set_entry_point("GENERATE")
graph.add_edge("REFLECT", "GENERATE")
graph.add_conditional_edges("GENERATE", should_continue)

In [29]:
app = graph.compile()


In [30]:
# Test the agent
question = "How will your project positively impact underserved communities?"
response = app.invoke([HumanMessage(content=question)])
print(response)

from langchain.callbacks.tracers.langchain import wait_for_all_tracers
wait_for_all_tracers()

[HumanMessage(content='How will your project positively impact underserved communities?', additional_kwargs={}, response_metadata={}, id='4b92bbb5-2198-4428-95fd-1f72e669d3ae'), AIMessage(content='Our project is specifically designed to address the needs of underserved communities and bring about positive change in their lives. Through our comprehensive approach, we will provide essential services, resources, and support to help empower individuals and families in these communities.\n\nOne key aspect of our project is to improve access to education and training opportunities, thereby equipping community members with the skills and knowledge needed to secure better job prospects and improve their economic situation. Additionally, we will facilitate access to healthcare services, mental health support, and other essential resources to address the specific needs of underserved populations.\n\nBy actively engaging with community leaders and stakeholders, we aim to foster a collaborative an

Langsmith link:https://smith.langchain.com/public/0edf7661-3288-451d-9d0a-6bea5efbc012/r